## `Alpha Vantage`

An API that provides comprehensive financial data for stocks, forex, cryptocurrencies, and technical indicators.
- **Advantage**: Offers multiple data categories, including intraday data at intervals.
- **Disadvantage**: Limited free tier; paid plans are required for high-frequency data access and larger query volumes.
    - **Update on 11/14: My results are null because I exceeded the 25 requests per day limit.**
  
Source: https://www.alphavantage.co/support/#api-key

### Step 0: Package

In [4]:
#pip install python-dotenv
import os
import requests
from dotenv import load_dotenv
from datetime import datetime

In [5]:
# Load environment variables from the .env file
load_dotenv()

# Access the API key
api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
#print(api_key)

### Step 1: Use `Alpha Vantage` to retrieve *daily* stock prices

In [7]:
# Function to get the market cap of a stock
def get_market_cap(ticker):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        market_cap = data.get("MarketCapitalization")
        return int(market_cap) if market_cap else 0
    else:
        print(f"Failed to retrieve data for {ticker}")
        return 0

# Main function to get the largest 10 stocks by market cap within a date range
def get_largest_sp500_stocks(num_stocks=10, start_date="2024-11-01", end_date="2024-11-05"):
    # Example list of S&P 500 tickers 
    sp500_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "TSLA", "BRK.B", "META", "UNH", "JPM"]
    
    # Dictionary to store ticker and market cap
    market_caps = {}
    
    # Fetch market cap for each ticker
    for ticker in sp500_tickers:
        market_cap = get_market_cap(ticker)
        market_caps[ticker] = market_cap
    
    # Sort tickers by market cap in descending order and select the top `num_stocks`
    largest_stocks = sorted(market_caps, key=market_caps.get, reverse=True)[:num_stocks]

    # Fetch historical data for each of the largest stocks
    historical_data = {}
    base_url = "https://www.alphavantage.co/query"
    for ticker in largest_stocks:
        params = {
            "function": "TIME_SERIES_DAILY",
            "symbol": ticker,
            "apikey": api_key
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json().get("Time Series (Daily)", {})
            # Filter by date range
            filtered_data = {
                date: values for date, values in data.items()
                if start_date <= date <= end_date
            }
            historical_data[ticker] = {
                "market_cap": market_caps[ticker],
                "history": filtered_data
            }
        else:
            print(f"Failed to retrieve historical data for {ticker}")
    
    return historical_data

# Call the function and display results
largest_stocks_hist = get_largest_sp500_stocks(start_date="2024-11-01", end_date="2024-11-05")
for ticker, data in largest_stocks_hist.items():
    print(f"--- {ticker} ---")
    print(f"Market Cap: ${data['market_cap']:,}")
    for date, values in data["history"].items():
        print(f"Date: {date}")
        print(f"Open: {values['1. open']}")
        print(f"High: {values['2. high']}")
        print(f"Low: {values['3. low']}")
        print(f"Close: {values['4. close']}")
        print(f"Volume: {values['5. volume']}\n")

--- AAPL ---
Market Cap: $0
--- MSFT ---
Market Cap: $0
--- GOOGL ---
Market Cap: $0
--- AMZN ---
Market Cap: $0
--- NVDA ---
Market Cap: $0
--- TSLA ---
Market Cap: $0
--- BRK.B ---
Market Cap: $0
--- META ---
Market Cap: $0
--- UNH ---
Market Cap: $0
--- JPM ---
Market Cap: $0


### Step 2: Use `Alpha Vantage` to retrieve *intraday* stock prices

In [9]:
# Function to get the market cap of a stock
def get_market_cap(ticker):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        market_cap = data.get("MarketCapitalization")
        return int(market_cap) if market_cap else 0
    else:
        print(f"Failed to retrieve data for {ticker}")
        return 0

# Main function to get the first 10 minutes of intraday data for the largest stocks by market cap
def get_largest_sp500_intraday(num_stocks=10, interval="5min", limit_intervals=2):
    
    # Example list of S&P 500 tickers
    #sp500_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "TSLA", "BRK.B", "META", "UNH", "JPM"]
    sp500_tickers = ["AAPL", "MSFT", "GOOGL"]
    
    # Dictionary to store ticker and market cap
    market_caps = {}
    
    # Fetch market cap for each ticker
    for ticker in sp500_tickers:
        market_cap = get_market_cap(ticker)
        market_caps[ticker] = market_cap
    
    # Sort tickers by market cap in descending order and select the top `num_stocks`
    largest_stocks = sorted(market_caps, key=market_caps.get, reverse=True)[:num_stocks]

    # Fetch intraday data for each of the largest stocks
    historical_data = {}
    base_url = "https://www.alphavantage.co/query"
    for ticker in largest_stocks:
        params = {
            "function": "TIME_SERIES_INTRADAY",
            "symbol": ticker,
            "interval": interval,  
            "apikey": api_key
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json().get(f"Time Series ({interval})", {})
            # Sort and limit to the first few intervals of the day (first 10 minutes)
            filtered_data = dict(list(data.items())[:limit_intervals])
            historical_data[ticker] = {
                "market_cap": market_caps[ticker],
                "history": filtered_data
            }
        else:
            print(f"Failed to retrieve intraday data for {ticker}")
    
    return historical_data

# Call the function and display results
largest_stocks_intraday = get_largest_sp500_intraday(interval="5min", limit_intervals=2)
for ticker, data in largest_stocks_intraday.items():
    print(f"--- {ticker} ---")
    print(f"Market Cap: ${data['market_cap']:,}")
    for timestamp, values in data["history"].items():
        print(f"Time: {timestamp}")
        print(f"Open: {values['1. open']}")
        print(f"High: {values['2. high']}")
        print(f"Low: {values['3. low']}")
        print(f"Close: {values['4. close']}")
        print(f"Volume: {values['5. volume']}\n")

--- AAPL ---
Market Cap: $0
--- MSFT ---
Market Cap: $0
--- GOOGL ---
Market Cap: $0
